# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_total = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity":"sum"})

In [5]:
df_total.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pivot_table = df_total.pivot_table(values ="Quantity", index='ProductName', columns='CustomerID', aggfunc= sum, fill_value=0)

In [7]:
pivot_table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_table.T, 'euclidean'))), 
                         index=pivot_table.columns, columns=pivot_table.columns)

In [9]:
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
similarities_33 = distances[33].sort_values(ascending=False)[1:]
similarities_33[:5]

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [11]:
df_clean = df_total.loc[[264,3535,3317,2503,3305]]
df_clean

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [12]:
df = df_clean.groupby(["ProductName"]).agg({"Quantity":"sum"})

In [13]:
df_final = df.sort_values(by='Quantity', ascending=False)

In [14]:
df_top = df_final.head()

In [15]:
df_top

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [16]:
customer = df_total.loc[33]

In [17]:
customer.head()

,Quantity
ProductName,
Apricots - Dried,1
Assorted Desserts,1
Bandage - Flexible Neon,1
"Bar Mix - Pina Colada, 355 Ml",1
"Beans - Kidney, Canned",1


In [18]:
df_merge = pd.concat((customer, df_top), axis=1)

In [19]:
df_merge.columns = ["Customer qtd", "Recommended qtd"]

In [20]:
df_merge = df_merge.fillna(0)

In [21]:
df_filter = df_merge[df_merge["Customer qtd"] == 0]

In [22]:
df_filter

,Customer qtd,Recommended qtd
Butter - Unsalted,0.0,3.0
Wine - Ej Gallo Sierra Valley,0.0,3.0
Soup - Campbells Bean Medley,0.0,3.0
Wine - Blue Nun Qualitatswein,0.0,3.0


In [23]:
product_recommendations = df_filter.index

In [24]:
product_recommendations

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
      dtype='object')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [88]:
rec = {}

In [26]:
lst = similarities_33[:5]

In [27]:
lst = lst.index
lst

Int64Index([264, 3535, 3317, 2503, 3305], dtype='int64', name='CustomerID')

In [30]:
lst_ids = [264, 3535, 3317, 2503, 3305]

In [58]:
def recommendations(id):
        a = distances[id].sort_values(ascending=False)[1:]
        a = a[:5]
        df = df_total.loc[[a.index[0], a.index[1], a.index[2], a.index[3], a.index[4]]]
        return df

In [63]:
def getDf(df):
    dff = df.groupby(["ProductName"]).agg({"Quantity":"sum"})
    dffinal = dff.sort_values(by='Quantity', ascending=False)
    df_top = dffinal.head()
    return df_top

In [69]:
def getCustomer(id):
    customer = df_total.loc[id]
    return customer

In [71]:
recommendations(264)
getDf(df)
getCustomer(264)

,Quantity
ProductName,
Apricots - Halves,1
Apricots Fresh,1
Bacardi Breezer - Tropical,1
Bagel - Plain,1
Banana - Leaves,1
...,...
Wine - Chardonnay South,1
Wine - Gato Negro Cabernet,1
Wine - Redchard Merritt,1


In [77]:
df_merge = pd.concat((customer, df_top), axis=1)
df_merge.columns = ["Customer qtd", "Recommended qtd"]
df_merge = df_merge.fillna(0)
df_filter = df_merge[df_merge["Customer qtd"] == 0]
product_recommendations_264 = df_filter.index
product_recommendations_264
    

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
      dtype='object')

In [89]:
rec.update( {'264' : product_recommendations_264} )

In [94]:
recommendations(3535)
getDf(df)
getCustomer(3535)
print('ok')

ok


In [76]:
df_merge = pd.concat((customer, df_top), axis=1)
df_merge.columns = ["Customer qtd", "Recommended qtd"]
df_merge = df_merge.fillna(0)
df_filter = df_merge[df_merge["Customer qtd"] == 0]
product_recommendations_3535 = df_filter.index
product_recommendations_3535

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
      dtype='object')

In [90]:
rec.update( {'3535' : product_recommendations_3535} )

In [95]:
recommendations(3317)
getDf(df)
getCustomer(3317)
print("ok")

ok


In [96]:
df_merge = pd.concat((customer, df_top), axis=1)
df_merge.columns = ["Customer qtd", "Recommended qtd"]
df_merge = df_merge.fillna(0)
df_filter = df_merge[df_merge["Customer qtd"] == 0]
product_recommendations_3317 = df_filter.index
product_recommendations_3317

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
      dtype='object')

In [97]:
rec.update( {'3317' : product_recommendations_3317} )

In [98]:
recommendations(2503)
getDf(df)
getCustomer(2503)
print("ok")

ok


In [99]:
df_merge = pd.concat((customer, df_top), axis=1)
df_merge.columns = ["Customer qtd", "Recommended qtd"]
df_merge = df_merge.fillna(0)
df_filter = df_merge[df_merge["Customer qtd"] == 0]
product_recommendations_2503 = df_filter.index
product_recommendations_2503

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
      dtype='object')

In [100]:
rec.update( {'2503' : product_recommendations_2503} )

In [101]:
recommendations(3305)
getDf(df)
getCustomer(3305)
print("ok")

ok


In [102]:
df_merge = pd.concat((customer, df_top), axis=1)
df_merge.columns = ["Customer qtd", "Recommended qtd"]
df_merge = df_merge.fillna(0)
df_filter = df_merge[df_merge["Customer qtd"] == 0]
product_recommendations_3305 = df_filter.index
product_recommendations_3305

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
      dtype='object')

In [103]:
rec.update( {'3305' : product_recommendations_3305} )

In [104]:
rec

{'264': Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
       dtype='object'),
 '3535': Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
       dtype='object'),
 '3317': Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
       dtype='object'),
 '2503': Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
       dtype='object'),
 '3305': Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein'],
       dtype='object')}

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [105]:
df_last = pd.DataFrame([[key, rec[key]] for key in rec.keys()], columns=['CustomerID', 'Recommended_Products'])

In [107]:
df_last

,CustomerID,Recommended_Products
0,264,"Index(['Butter - Unsalted', 'Wine - Ej Gallo S..."
1,3535,"Index(['Butter - Unsalted', 'Wine - Ej Gallo S..."
2,3317,"Index(['Butter - Unsalted', 'Wine - Ej Gallo S..."
3,2503,"Index(['Butter - Unsalted', 'Wine - Ej Gallo S..."
4,3305,"Index(['Butter - Unsalted', 'Wine - Ej Gallo S..."


In [ ]:
# Tengo que abrir los productos de Recommended Product, pero voy sin tiempo y todavia tengo el otro lab. 
#¿Como puedo hacer esa separacion?


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
#tengo que empezar el otro lab, luego si me da tiempo hago este. 